In [1]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [2]:

# Load the Falcon model and tokenizer
model_name = "tiiuae/falcon-7b"  # Example model, adjust based on available model and resources
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/mnt/Data/prabirmondal/anaconda3/envs/alyona/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [26]:
import torch
torch.cuda.empty_cache()

In [27]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [28]:

# Directories for input and output
input_dir = '/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/yuvraj/questions/popular/hop1/YES_NO_cleaned'
output_dir = '/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/alyona/results'
os.makedirs(output_dir, exist_ok=True)


In [29]:

# Function to filter Falcon responses
def filter(response):
    # Split response into separate lines to find each instruction-answer pair
    lines = response.lower().splitlines()
    
    # Initialize a variable to hold the last valid answer
    final_answer = None
    
    # Iterate through lines to find 'yes' or 'no' answers
    for line in lines:
        line = line.strip()  # Remove whitespace
        # Check if the line is exactly 'yes' or 'no' and update the final answer
        if line == "yes" or line == "no":
            final_answer = line

    # Return the last valid 'yes' or 'no' found, or None if not found
    return final_answer


In [32]:

# Function to generate Falcon response for a given question
def generate_response(question):
    prompt = f'''
        [INST] <<SYS>>
        You have a good knowledge of Hollywood movies. You will be given a question
        about a particular Hollywood movie. Using your knowledge, Give answer as yes or no only for the given question.
        Don't let the previous responses affect this response.
        <</SYS>>
        Was 'A Haunted House 2 2014' confirmed to be released on March 28, 2014? [/INST]
        yes
        [INST] Did the principal photography and production for 'A Haunted House 2 2014' begin in 2015? [/INST]
        no
        [INST] {question} [/INST]
    '''
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=50, temperature=0.7, do_sample=True)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
    return filter(response)


In [33]:

# Define custom accuracy calculation function
def calculate_accuracy(true_answers, predictions):
    correct = sum(1 for true, pred in zip(true_answers, predictions) if true == pred)
    return correct / len(true_answers) if true_answers else 0

type_accuracies = {}
overall_accuracies = []

for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        input_file_path = os.path.join(input_dir, filename)
        output_file_path = os.path.join(output_dir, filename)
        
        # Extract the type from the filename
        file_type = filename.split('_')[-1].split('.')[0].lower()

        # Check if the output file already exists
        if os.path.exists(output_file_path):
            # If already processed, read the last accuracy value directly
            existing_data = pd.read_csv(output_file_path)
            file_accuracy = float(existing_data.iloc[-1, 2])  # Assuming last row, third column stores the accuracy
            overall_accuracies.append(file_accuracy)
            
            if file_type not in type_accuracies:
                type_accuracies[file_type] = []
            type_accuracies[file_type].append(file_accuracy)
            
            # Skip further processing for this file
            print(f"Skipping already processed file: {filename}, Accuracy: {file_accuracy}")
            continue

        # Read the input data and make predictions
        data = pd.read_csv(input_file_path)
        predictions = []
        true_answers = data['Answer'].tolist()
        
        for question in tqdm(data['Question'], desc=f"Processing {filename}"):
            response = generate_response(question)
            predictions.append(response)
        
        # Calculate accuracy and add to overall and type-specific lists
        file_accuracy = calculate_accuracy(true_answers, predictions)
        overall_accuracies.append(file_accuracy)
        
        if file_type not in type_accuracies:
            type_accuracies[file_type] = []
        type_accuracies[file_type].append(file_accuracy)

        # Save predictions and accuracy to CSV
        data['FALCON_response'] = predictions
        data.loc[len(data)] = ["", "", file_accuracy]  # Assuming the last column is for accuracy
        data.to_csv(output_file_path, index=False)


Processing The Lord of the Rings_ The Return of the King_2003_Cast.csv:   0%|          | 0/96 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing The Lord of the Rings_ The Return of the King_2003_Cast.csv:   0%|          | 0/96 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 17.19 MiB is free. Process 13604 has 6.71 GiB memory in use. Process 35176 has 6.36 GiB memory in use. Process 12298 has 494.00 MiB memory in use. Process 39460 has 392.00 MiB memory in use. Including non-PyTorch memory, this process has 9.73 GiB memory in use. Of the allocated memory 8.91 GiB is allocated by PyTorch, and 25.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

# Calculate average accuracy per type and overall accuracy
report_content = []
for file_type, accuracies in type_accuracies.items():
    avg_type_accuracy = sum(accuracies) / len(accuracies)
    report_content.append(f"{file_type.capitalize()} Accuracy: {avg_type_accuracy:.2f}")

overall_accuracy = sum(overall_accuracies) / len(overall_accuracies) if overall_accuracies else 0
report_content.append(f"Overall Accuracy: {overall_accuracy:.2f}")

# Write the report to file
with open(report_path, 'w') as f:
    f.write("\n".join(report_content))

print("Processing complete. Check the output directory for results and report.txt.")


In [13]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available. {torch.cuda.device_count()} device(s) detected.")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Using CPU.")


CUDA is available. 2 device(s) detected.
Device 0: NVIDIA GeForce RTX 3090
Device 1: NVIDIA GeForce RTX 3090
